## Simulation as Optimization: particle simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from celluloid import Camera
from IPython.display import HTML
from base64 import b64encode

In [ ]:
def make_video(xs, path, interval=60, **kwargs): # xs: [time, N, 2]
    fig = plt.gcf() ; fig.set_dpi(100) ; fig.set_size_inches(3, 3)
    camera = Camera(fig)
    for i in range(xs.shape[0]):
        plt.plot(xs[i][...,0], xs[i][...,1], 'k.', markersize=20)
        plt.axis('equal') ; plt.xlim(0,1) ; plt.ylim(0,1)
#         plt.xticks([], []); plt.yticks([], [])
        camera.snap()
    anim = camera.animate(blit=True, interval=interval, **kwargs)
    anim.save(path) ; plt.close()

## Get a baseline simulation working

In [ ]:
N = 15
dt = 1
np.random.seed(0)
x0 = np.random.rand(N,2)*.9 + 0.05
v0 = np.random.randn(N,2).clip(-1,1)*.02
x1 = x0 + dt*v0

In [ ]:
def particle_numerical(x0, x1, dt, steps=20, box_width=1):
    xs = [x0, x1]
    ts = [0, dt]
    v = (x1 - x0) / dt
    x = xs[-1]
    for i in range(steps-2):
        a = 0 # get forces/accelerations
        v = v + a*dt
        x = x + v*dt
        x = x % box_width
        xs.append(x)
        ts.append(ts[-1]+dt)
    return np.asarray(ts), np.stack(xs)

t_num, x_num = particle_numerical(x0, x1, dt)

In [ ]:
xs = x_num
path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Recover the same dynamics by minimizing the action

In [ ]:
def lagrangian(q, m=1, g=1):
    (x, xdot) = q
    return .5*m*xdot**2 #- m*g*x
  
def action(x, dt=1):
    dx = x[1:] - x[:-1]
    wraps = (dx.abs() > 0.9)
    dx = -dx.sign()*(dx.abs()-1)*wraps + dx*(~wraps)
    xdot = (dx) / dt
    xdot = torch.cat([xdot, xdot[-1:]], axis=0)
    return lagrangian(q=(x, xdot)).sum()

def get_path_between(x, steps=1000, step_size=1e-1, dt=1, box_width=1):
    t = np.linspace(0, len(x)-1, len(x)) * dt
    xs = [x.clone().data]
    for i in range(steps):
        grad = torch.autograd.grad(action(x, dt), x)
        grad_x = grad[0]
        grad_x[[0,-1]] *= 0  # fix first and last coordinates by zeroing their grads
        x.data -= grad_x * step_size
        
        x = x % box_width # x is subject to modulo arithmetic

        if i % (steps//15) == 0:
            xs.append(x.clone().data)
            print('step={:04d}, S={:.3e}'.format(i, action(x).item()))
    return t, x, xs

N = 15
dt = 1
np.random.seed(0)
x0 = np.random.rand(N,2)*.9 + 0.05
v0 = np.random.randn(N,2).clip(-1,1)*.02
x1 = x0 + dt*v0
t_num, x_num = particle_numerical(x0, x1, dt)

x_noise = .06*np.random.randn(*x_num.shape).clip(-1,1)
x_noise[:1] = x_noise[-1:] = 0
x_pert = (x_num + x_noise).reshape(-1, N*2)
# x_pert = x_num.reshape(-1, N*2)
# x_pert = x_pert + 1#np.random.randn(*x_pert[1:-1].shape)
x0 = torch.tensor(x_pert, requires_grad=True) # [time, N*2]
t_min, x_min, xs_min = get_path_between(x0)

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
# xs = x_num

path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)

path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)